In [ ]:
import os
import glob
from music21 import converter, key, interval, pitch, midi

# CONFIGURATION

input_dir = "/home/cepatinog/smc-assignments/final_project/my_jazz_project/data/midi_files/WjazzD"          # <- Ajusta esta ruta
output_dir = "/home/cepatinog/smc-assignments/final_project/my_jazz_project/data/midi_files/WjazzD_C_Am"  # <- Carpeta de salida

os.makedirs(output_dir, exist_ok=True)

MIN_PITCH = 48
MAX_PITCH = 95

def get_pitch_range(part):
    """Returns (min, max) pitch considering Notes and Chords."""
    pitches = []
    for n in part.flatten().notes:
        if n.isNote:
            pitches.append(n.pitch.midi)
        elif n.isChord:
            top_note = max(n.pitches, key=lambda p: p.midi)
            pitches.append(top_note.midi)
    return (min(pitches), max(pitches)) if pitches else (None, None)

def transpose_to_c_or_am(score):
    original_key = score.analyze('key')
    print(f"  Detected key: {original_key.tonic.name} {original_key.mode}")

    target_key = key.Key('C') if original_key.mode == 'major' else key.Key('A')
    i = interval.Interval(original_key.tonic, target_key.tonic)
    transposed = score.transpose(i)

    print(f"  Transposed by {i.niceName}")
    return transposed, i

def adjust_octaves(score):
    part = score.parts[0]
    min_pitch, max_pitch = get_pitch_range(part)

    if min_pitch is None:
        print("  ⚠️ No pitches found.")
        return score

    print(f"  Pitch range after tonal transposition: {min_pitch}–{max_pitch}")

    shift = 0
    while min_pitch + shift < MIN_PITCH:
        shift += 12
    while max_pitch + shift > MAX_PITCH:
        shift -= 12

    print(f"  Octave shift applied: {shift:+} semitonos")

    if shift != 0:
        score = score.transpose(shift)

    return score

# PROCESS ALL FILES

midi_files = glob.glob(os.path.join(input_dir, "**/*.mid"), recursive=True)
print(f"\n MIDI files found: {len(midi_files)}")

for path in midi_files:
    try:
        filename = os.path.basename(path)
        print(f"\n🎵 Processing {filename}")

        score = converter.parse(path)

        # Original range 
        original_min, original_max = get_pitch_range(score.parts[0])
        print(f"  Original range: {original_min}–{original_max}")

        # Harmonic transposition + octave adjustment
        score, i = transpose_to_c_or_am(score)
        score = adjust_octaves(score)

        # File name with transposition info
        semitones = i.chromatic.directed
        new_name = filename.replace(".mid", f"_shift{semitones:+}.mid")
        out_path = os.path.join(output_dir, new_name)

        # Save transposed file
        mf = midi.translate.music21ObjectToMidiFile(score)
        mf.open(out_path, 'wb')
        mf.write()
        mf.close()

        print(f"  ✅ Saved: {out_path}")

    except Exception as e:
        print(f"  ❌ Error processing {filename}: {e}")




🎯 Archivos MIDI encontrados: 441

🎵 Procesando MichaelBrecker_DeltaCityBlues_FINAL.mid
  Rango original: 44–77
  Detected key: E- major
  Transposed by Minor Third
  Pitch range after tonal transposition: 41–74
  Octave shift applied: +12 semitonos
  ✅ Guardado: /home/cepatinog/smc-assignments/final_project/my_jazz_project/data/midi_files/WjazzD_C_Am/MichaelBrecker_DeltaCityBlues_FINAL_shift-3.mid

🎵 Procesando RoyEldridge_BodyAndSoul_FINAL.mid
  Rango original: 55–80
  Detected key: B- minor
  Transposed by Minor Second
  Pitch range after tonal transposition: 54–79
  Octave shift applied: +0 semitonos
  ✅ Guardado: /home/cepatinog/smc-assignments/final_project/my_jazz_project/data/midi_files/WjazzD_C_Am/RoyEldridge_BodyAndSoul_FINAL_shift-1.mid

🎵 Procesando JoeLovano_ICan'tGetStarted_FINAL.mid
  Rango original: 44–78
  Detected key: C major
  Transposed by Perfect Unison
  Pitch range after tonal transposition: 44–78
  Octave shift applied: +12 semitonos
  ✅ Guardado: /home/cepatin